# Evoked .fif files display

This notebook shows how to display evoked data stored in .fif files.

This application is relying on the [mne](https://mne.tools/stable/index.html)
software.

Upload an .fif file containing evoked data.

There are some example files you can upload in the [GitHub repository](https://github.com/agramfort/mne-web-apps/tree/master/example-fif-evoked).

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.ioff()

In [3]:
# imports
from traitlets import dlink
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO
import mne

# widgets
file_picker = widgets.FileUpload(accept='*.fif')
file_name = widgets.Text()
#conditions = ["0", "1"]
#conditions_rb = widgets.RadioButtons(
#    options=conditions,
#    description='Condition:',
#    disabled=False,
#)

#vbox = widgets.VBox([widgets.HBox([file_picker, file_name]), conditions_rb])
vbox = widgets.VBox([widgets.HBox([file_picker, file_name])])

# display evoked file
def show_evoked(file_content, condition=0):
    # bridge: FileUpload - stl.mesh
    content = next(iter(file_content.values()))['content']
    
    f = open('/tmp/temp-ave.fif', 'wb')
    f.write(content)
    f.close()
    evoked = mne.read_evokeds('/tmp/temp-ave.fif', baseline=(None, 0))
    fig = evoked[condition].plot(selectable=False, show=False)
    return fig.canvas

# get filename from FileUpload dict
def get_name(file_upload):
    if len(file_upload) == 0:
        return ''
    return next(iter(file_upload.values()))['metadata']['name']

def change_input(change):
    global vbox, file_picker, file_name, conditions_rb
#    myevoked = show_evoked(file_picker.value, conditions_rb.value)
    myevoked = show_evoked(file_picker.value)
#    vbox.children = (widgets.HBox([file_picker, file_name]), conditions_rb, myevoked)
    vbox.children = (widgets.HBox([file_picker, file_name]), myevoked)

# links between widgets
#conditions_rb.observe(change_input, 'value')
dlink((file_picker, 'value'), (file_name, 'value'), get_name)
file_picker.observe(change_input, 'value')

vbox